## MicroPython ESP32 Experimentation

### Establishing connection to target board
First, make sure you've got the right serial port. On unix-based systems, you can run `ls /dev/tty.*` to see your available serial devices. Replace as necessary below.

This will allow Jupyter (your host computer) to run commands and send/receive information to/from your target board in real time using the MicroPython REPL.

In [61]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ["b'\\xfcets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode:DIO, clock div:2\\r\\nload:0x3fff0018,len:4\\r\\nload:0x3fff001c,len:5204\\r\\nload:0x40078000,len:12136\\r\\nload:0x40080400,len:3496\\r\\nentry 0x4008063c\\r\\nets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode:DIO, clock div:2\\r\\nload:0x3fff0018,len:4\\r\\nload:0x3fff001c,len:5204\\r\\nload:0x40078000,len:12136\\r\\nload:0x40080400,l\\x80\\xe5\\x810x4008063c\\r\\nets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x10 (RTCWDT_RTC_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode:DIO, clock div:2\\r\\nloa

In [10]:
%sendtofile lib/runnera.py --source lib/runner.py

Sent 246 lines (8210 bytes) to lib/runnera.py.


In [11]:
%sendtofile lib/synthetica.py --source lib/synthetic.py

Sent 47 lines (1573 bytes) to lib/synthetica.py.


In [12]:
%sendtofile lib/decodinga.py --source lib/decoding.py

Sent 238 lines (8870 bytes) to lib/decodinga.py.


## Using a Runner for experimentation and logging
The a `Runner` is encapsulates the core functions in this EEG system, including peripheral setup, sampling, signal processing, logging and memory management. The `OnlineRunner` offers mostly the same functionality as the standard `Runner` class, except it allows for logging and other communication with a remote server - either on the Internet or on your local network.

### Offline functionality
The standard `Runner` is good for testing core functionality without the need for remote logging. See below for initialisation and execution.

In [31]:
from lib.runner import Runner

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup() # setup peripherals and memory buffers

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498


### Calibration
If you are using an algorithm that leverages calibration data (MsetCCA, GCCA), you will need to record some calibration data to fit the decoder model. This is usually only done once off before inference starts. You may want to recalibrate at some semi-regular interval too though. 

At the moment, there is not an integrated process to record calibration data in the `Runner` class. You have to record calibration data and provide it to the runner which it will in turn use to fit its internal decoder model. In future, this will hopefully become more integrated and easy. For now, some random calibration data is generated below to illustrate the format which the runner/decoder expects. You need to provide iid calibration data trials for each stimulus frequency.

Note that if you try to run calibration using an incompatible algorithm (such as standard CCA), a warning will be generated and the calibration sequence will be skipped.

In [29]:
print(gc.collect())

None


In [33]:
from lib.synthetic import synth_X

calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}
runner.calibrate(calibration_data)

In [62]:
print(gc.mem_free())

110224


### Decoding
When configured with a set of stimulus frequencies $\mathcal{F}=\{f_1, \dots, f_k, \dots, f_K\}$, the `Runner`'s decoder model consists of $K$ independent sub-classifiers $\Phi_k$ that each leverage the decoding algorithm selected. These independent classifiers must be calibrated independently. When the `Runner` is presented a new test observation, each sub-classifier $\Phi_k$ produces an output correlation estimate corresponding to $f_k$. Ultimately, the runner outputs a dictionary of frequency-correlation pairs of the form
```python
{f_1: 0.12, f_2: 0.03, f_3: 0.85}
```
The decoded output frequency is the one corresponding to the largest correlation in this output dictionary. In this example, it would be $f_3$.

In [39]:
test_freq = 7 # 7 Hz test signal
test_data = synth_X(test_freq, Nc, Ns, Nt=1)

print(runner.decoder.classify(test_data))

{12: 0.004733905, 10: 0.01099741, 7: 0.9917706}


### Asynchronous operation
Once the `Runner` has been configured and calibrated (if applicable), its internal `run()` loop can be started in which it will asynchronously sample and decode EEG data at preconfigured frequencies. Timing is handled using hardware timers on the ESP32 and interrupts are used to run asynchronous ISRs that handle sampling, preprocessing, filtering and decoding.

Note that once the async run loop has begun, you can still run commands or view the `Runner`'s attributes although there may be a noticeable delay since ISRs will typically get higher execution priority and there are quite a few interrupt loops running.

In [41]:
# start sampling and recording data (logging not setup in this case)
runner.run()

In [42]:
# see if runner has indeed started smapling
print(runner.is_sampling)

True


In [43]:
# display the contents of the output buffer - this will be updated internally by the runner
# at a rate determined by the sampling frequency and sample buffer size (typically every 1s)
print(runner.output_buffer)

[-0.001441792, -0.3037838, -1.551759, -0.219501, 3.194653, -1.072062, -0.200694, 6.702996, -1.947414, -5.027199, 1.564445, 2.672828, 1.21374, -1.437051, 2.925138, -1.231109, -6.948754, 2.045549, 4.036036, -2.170489, 0.03711551, 4.074638, -3.130945, -4.147216, 4.156588, 4.188887, -1.814264, -0.1168619, 2.935409, -4.270164, -3.49786, 3.961716, 3.079408, -3.522197, -4.539949, -3.970249, -0.990411, 7.356385, -0.0371345, -2.75865, 2.164335, -1.930601, -4.593678, 0.5314817, 6.744837, -3.217182, -5.489737, -0.693433, -3.941568, -1.549129, 3.282845, 1.359998, -4.587076, 3.524699, 2.54799, -5.847449, 1.876395, 4.986438, 2.018594, -2.166026, 3.279232, 5.460333, -2.510297, -0.7549546, -0.002938264, -0.4208137, -2.996173, -5.388202, 0.4557866, 6.510355, 0.5805722, -2.615711, 1.715585, -0.9709125, -5.372795, 1.403303, 6.322515, -2.846665, -5.36191, -0.8382293, -3.404387, -1.949912, 3.903792, 1.038809, -4.176731, 3.496768, 2.636908, -5.576463, 1.728363, 5.407993, 1.815191, -1.779266, 3.18501, 5.6863

In [46]:
# decode the contents of the output buffer. There will be a delay here if the runner 
# is currently running (i.e. `is_sampling=True`).
print(runner.decode())

{12: 0.0095, 10: 0.02259, 7: 0.02649}


In [47]:
# stop runner
runner.stop()

#### Simple decoding loop
In order to test online decoding, here is a basic synchronous loop-based option. Interrupt the cell to stop the infinite loop.

In [2]:
import utime as time
from lib.runner import Runner

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'

decode_period_s = 2 # read decoded output every x seconds

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup()

if decoding_algo in ['MsetCCA', 'GCCA']:
    from lib.synthetic import synth_X

    calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}
    runner.calibrate(calibration_data)

runner.run() # start async run loop

try:
    while True:
        time.sleep(decode_period_s)
        print(runner.decoded_output)
except KeyboardInterrupt:
    runner.stop()
    print('received SIGINT - stopping')

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62497708393011
{}
.{12: 0.15038, 10: 0.02075, 7: 0.11349}
{12: 0.15038, 10: 0.02075, 7: 0.11349}
.{12: 0.04253, 10: 0.02158, 7: 0.00613}
{12: 0.04253, 10: 0.02158, 7: 0.00613}
{12: 0.05438, 10: 0.00615, 7: 0.08}
.{12: 0.05438, 10: 0.00615, 7: 0.08}
{12: 0.0278, 10: 0.00456, 7: 0.02368}


*** Sending Ctrl-C

received SIGINT - stopping


### Testing your WiFi connection
In order to connect to a local WiFi network, you'll need to supply your network SSID and password in a `.env` file on the board. Doing this is easy: 
1. On your computer, create a `.env` file using `touch .env`. Update the `.env` file with the required fields:
    
    ```bash
    #.env 
    WIFI_SSID=<your network name>
    WIFI_PASSWORD=<your network password>
    
    ```
    
2. Send this file to your target device using the following command:
    ```ipython
%sendtofile --source lib/.env lib/.env  --binary
```

You may need to update the local (source) path to your `.env` file depending on where you created/stored it.

In [48]:
%sendtofile --source /Users/rishil/Desktop/.env lib/.env  --binary

Sent 55 bytes in 2 chunks to lib/.env.


In [67]:
from lib.utils import connect_wifi, load_env_vars
from lib.synthetic import synth_X

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'
calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

network config: ('192.168.0.68', '255.255.255.0', '192.168.0.1', '194.168.4.100')


#### Online Runner
Now that you've established network connectivitiy, you can test out an `OnlineRunner`. In order to test web logging to a remote server, we can use a basic HTTP logger. However, this obviously needs an API/server willing to accept our requests. There is a basic logging API using `Flask` in `/eeg_lib/logging_server.py`. You can run it using `python logging_server.py` which will spin up a development server on the predefined port (5000 or 5001). Then, just configure your `OnlineRunner` with the appropriate logger params and you're set.

In [68]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

api_host = "http://192.168.0.13:5001/" # make sure the port corresponds to your logging server configuration
log_params = dict(server=api_host, log_period=4, logger_type=logger_types.HTTP, send_raw=True, session_id='test_session_1')

runner = OnlineRunner(decoding_algo, buffer_size=Ns)
runner.setup(**log_params)

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
network config: ('192.168.0.68', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [69]:
# start the runner - you should see requests being made to your local server
runner.calibrate(calibration_data)
runner.run()

In [70]:
runner.stop()

## Experimentation

In [155]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython d8a7bf8-dirty on 2022-02-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython d8a7bf8-dirty on 2022-02-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [12]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute] [--source [SOURCE]] [--quiet]
                   [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US